Import libraries

In [10]:
import pandas as pd
import matplotlib as mpl
import sqlite3 as sq

Extract the dataset

In [11]:
earthquake_df = pd.read_csv(r'../data/all_earthquakes_by_month.csv')

Analyze the dataset

In [12]:
print(earthquake_df.head())
print(earthquake_df.shape)
print(earthquake_df.columns)

                       time   latitude   longitude  depth  mag magType  nst  \
0  2025-11-13T00:31:55.930Z  59.728500 -152.003300  48.90  1.6      ml  NaN   
1  2025-11-13T00:24:29.126Z  64.738100 -149.119300  16.70  1.5      ml  NaN   
2  2025-11-13T00:21:27.261Z  64.929100 -147.755700   0.00  1.2      ml  NaN   
3  2025-11-13T00:19:21.141Z  59.719100 -151.184600  25.70  2.2      ml  NaN   
4  2025-11-13T00:15:44.040Z  38.824833 -122.801498   2.58  0.7      md  8.0   

    gap      dmin   rms  ...                   updated  \
0   NaN       NaN  0.24  ...  2025-11-13T00:33:33.056Z   
1   NaN       NaN  0.76  ...  2025-11-13T00:26:37.074Z   
2   NaN       NaN  1.04  ...  2025-11-13T00:23:09.972Z   
3   NaN       NaN  0.95  ...  2025-11-13T00:31:12.108Z   
4  86.0  0.006851  0.02  ...  2025-11-13T00:32:17.204Z   

                               place        type horizontalError depthError  \
0  11 km WSW of Anchor Point, Alaska  earthquake             NaN       2.20   
1  15 km N of Four

Clean Dataset

In [13]:
print(earthquake_df.isna().sum())

#Columns with 1000+ NAs can be dropped since they are not needed for project
columns_to_drop = earthquake_df[earthquake_df.columns.difference(['time', 'latitude', 'longitude', 'depth', 'mag', 'magType', 'place', 'type'])]
clean_earthquake_df = earthquake_df.drop(columns=columns_to_drop)


#Only one NA for both mag and magtype so we can drop those rows
clean_earthquake_df.dropna(subset=['mag', 'magType'], inplace=True)

print(clean_earthquake_df.isna().sum()) #Check NAs for required columns

time                  0
latitude              0
longitude             0
depth                 0
mag                   1
magType               1
nst                1184
gap                1184
dmin               1185
rms                   2
net                   0
id                    0
updated               0
place                 0
type                  0
horizontalError    1494
depthError            2
magError           1216
magNst             1185
status                0
locationSource        0
magSource             0
dtype: int64
time         0
latitude     0
longitude    0
depth        0
mag          0
magType      0
place        0
type         0
dtype: int64


Check to make sure min and max values for depth and mag are not outliers

In [14]:
print(clean_earthquake_df['depth'].max())
print(clean_earthquake_df['depth'].min())

print(clean_earthquake_df['mag'].max())
print(clean_earthquake_df['mag'].min())

663.008
-3.5
6.8
-1.33


Remove Irrelevent Rows from Dataset

In [15]:
print(clean_earthquake_df['type'].unique().sum())

clean_earthquake_df[clean_earthquake_df['type'] == 'quarry blast']
clean_earthquake_df[clean_earthquake_df['type'] == 'explosion']
clean_earthquake_df[clean_earthquake_df['type'] == 'sonic boom']
clean_earthquake_df[clean_earthquake_df['type'] == 'ice quake']

#We only want to keep rows where type is earthquake
types_to_drop = ['quarry blast', 'explosion', 'sonic boom', 'ice quake']
clean_earthquake_df = clean_earthquake_df[~clean_earthquake_df['type'].isin(types_to_drop)]

print(clean_earthquake_df['type'].unique().sum())

print(clean_earthquake_df.shape) #Check how many rows lost


earthquakequarry blastexplosionsonic boomice quake
earthquake
(7155, 8)


Create Region Column

In [16]:
#Create region column
clean_earthquake_df.insert(7, "region", clean_earthquake_df['place'].apply(lambda x: x.split()[-1]))

#Change CA to California for consistency
clean_earthquake_df['region'] = clean_earthquake_df['region'].replace('CA', 'California')

#Print dataframe to check
print(clean_earthquake_df['region'].unique())
clean_earthquake_df

['Alaska' 'California' 'Tonga' 'Somalia' 'Nevada' 'Indonesia' 'Montana'
 'Mexico' 'Rico' 'Japan' 'Texas' 'Hawaii' 'Washington' 'Utah' 'Cyprus'
 'Wyoming' 'Chile' 'region' 'Philippines' 'Guinea' 'Idaho' 'Oregon' 'Sea'
 'Island' 'Russia' 'Oklahoma' 'Arizona' 'China' 'Uzbekistan' 'Micronesia'
 'Islands' 'Argentina' 'Hampshire' 'Republic' 'Peru' 'Barbados' 'Greece'
 'Turkey' 'Leste' 'Kansas' 'Guadeloupe' 'Zealand' 'Ridge' 'MX' 'Taiwan'
 'Carolina' 'Guam' 'Azerbaijan' 'Canada' 'Vanuatu' 'Maine' 'Tennessee'
 'India' 'Rise' 'Salvador' 'Georgia' 'Panama' 'Fiji' 'Ecuador' 'Iran'
 'Guatemala' 'America' 'Afghanistan' 'Tajikistan' 'Emirates' 'Arkansas'
 'Nepal' 'Futuna' 'Colorado' 'Passage' 'Armenia' 'Missouri' 'Pennsylvania'
 'Iraq' 'Ethiopia' 'Alabama' 'Bermuda' 'Iceland' 'Montenegro' '(Myanmar)'
 'Massachusetts' 'Maryland' 'Rica' 'Tobago' 'Colombia' 'Italy' 'York'
 'Poland' 'Mongolia' 'Barbuda' 'Sudan' 'Jersey' 'Kingdom' 'Malta'
 'Pakistan' 'Albania' 'Algeria' 'Libya' 'Nicaragua' 'Honduras' 'Sa

,time,latitude,longitude,depth,mag,magType,place,region,type
0,2025-11-13T00:31:55.930Z,59.728500,-152.003300,48.900,1.60,ml,"11 km WSW of Anchor Point, Alaska",Alaska,earthquake
1,2025-11-13T00:24:29.126Z,64.738100,-149.119300,16.700,1.50,ml,"15 km N of Four Mile Road, Alaska",Alaska,earthquake
2,2025-11-13T00:21:27.261Z,64.929100,-147.755700,0.000,1.20,ml,"3 km NW of Farmers Loop, Alaska",Alaska,earthquake
3,2025-11-13T00:19:21.141Z,59.719100,-151.184600,25.700,2.20,ml,"6 km ESE of Fritz Creek, Alaska",Alaska,earthquake
4,2025-11-13T00:15:44.040Z,38.824833,-122.801498,2.580,0.70,md,"7 km NW of The Geysers, CA",California,earthquake
...,...,...,...,...,...,...,...,...,...
7315,2025-10-14T00:47:40.463Z,13.950400,-91.225700,62.751,4.00,mb,"23 km SW of La Gomera, Guatemala",Guatemala,earthquake
7316,2025-10-14T00:44:26.502Z,10.798300,123.994700,10.000,4.40,mb,"2 km SSE of Lugo, Philippines",Philippines,earthquake
7317,2025-10-14T00:40:29.080Z,38.627666,-119.795670,-0.710,2.37,md,"7 km S of Markleeville, CA",California,earthquake
7318,2025-10-14T00:39:35.300Z,39.482333,-111.390333,1.730,1.05,ml,"9 km E of Spring City, Utah",Utah,earthquake


Print all unique values for each column

In [8]:
print(clean_earthquake_df.nunique())

for col in clean_earthquake_df.columns:
    print(clean_earthquake_df[col].unique())


time         7154
latitude     6060
longitude    6314
depth        3773
mag           480
magType         8
place        4249
region        105
type            1
dtype: int64
['2025-11-13T00:31:55.930Z' '2025-11-13T00:24:29.126Z'
 '2025-11-13T00:21:27.261Z' ... '2025-10-14T00:40:29.080Z'
 '2025-10-14T00:39:35.300Z' '2025-10-14T00:39:17.080Z']
[59.7285     64.7381     64.9291     ... 38.62766647 39.48233333
 40.36199951]
[-152.0033     -149.1193     -147.7557     ... -119.79566956 -111.39033333
 -124.33733368]
[48.9        16.7         0.         ... 62.751      -0.70999998
 26.15999985]
[ 1.6         1.5         1.2         2.2         0.7         1.26
  0.72        1.3         0.84        4.4         1.8         1.09
  4.6         2.12        0.3         1.1         1.97        1.9
  0.26        1.23        0.87        1.51        1.          1.05
  2.51        3.35        0.45        0.8         2.9         1.19
  1.77        1.33        1.39        0.75        0.77        2.
  0.9  

Load Data to Database

In [ ]:
#Create/Connect to local SQLite file
conn = sq.connect('../data/earthquakes.db')

#Write dataset to SQL table
clean_earthquake_df.to_sql('earthquakes', conn, if_exists='replace', index=False)

#Close the connection
conn.close()